In [0]:
%run "../includes/configuration"

In [0]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
demo_df = race_results.filter("race_year = 2020")

In [0]:
display(demo_df)

In [0]:
from pyspark.sql.functions import count, countDistinct, sum 

In [0]:
demo_df.select(count("*")).show()



In [0]:
demo_df.select(count("race_name")).show()

In [0]:
demo_df.select(countDistinct("race_name")).show()

In [0]:
demo_df.select(sum("points")).show()

In [0]:
demo_df.filter("driver_name = 'Lewis Hamilton'").select(sum("points")).show()

In [0]:
demo_df.filter("driver_name = 'Lewis Hamilton'").select(sum("points")), countDistinct("race_name")) \
    .withColumnRenamed("sum(points)", "total_points") \
    .withColumnRenamed("count(DISTINCT race_name)", "number_of_races") \
    .show()


###Group Aggregation

In [0]:
demo_df.groupBy("driver_name") \
    .sum("points") \
    .show()

In [0]:
demo_df.groupBy("driver_name") \
    .agg(sum("points").alias("total_points"), \
    countDistinct("race_name").alias("number_of_races")) \
    .show()

### Window Functions

In [0]:
demo_df = race_results_df.filter("race_year = (2019, 2020)") 
 

In [0]:
   demo_grouped_df = demo_df.groupBy("race_year","driver_name") \
   .agg(sum("points").alias("total_points"), countDistinct("race_name").alias("number_of_races")) \
   .show()

In [0]:
from pyspark.sql.functions import rank, desc
from pyspark.sql.window import Window

driverRankSpec = Window.partitionBy("race_year").orderBy(desc("total_points"))
demo_grouped_df.withColumn("rank", rank().over(driverRankSpec)) \
.show(100)